In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.compat.v1 import keras

config = ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement=True

keras.backend.set_session(InteractiveSession(config=config))
session = InteractiveSession(config=config)

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator #Augmentation
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)
                     #y_train為(50000,1)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train)
        std = np.std(X_train)
        X_train = (X_train-mean)/(std+1e-7)  #1e-7為1*10的-7次方,這是為了防止當std=0時會除以0
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()    # y_train.shape變為(50000,10)
y_test = one_hot.transform(y_test).toarray()


classifier = Sequential()

#卷積組合
classifier.add(Conv2D(32, (3,3), input_shape=(32,32,3), activation='relu'))
classifier.add(BatchNormalization())   # BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Conv2D(32, (3,3), activation='relu'))
classifier.add(BatchNormalization())  # BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, 
                     activation='relu',
                     kernel_regularizer=regularizers.l2(l=0.001))) ##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(p=0.5)) ##Dropout

classifier.add(Dense(output_dim=100,
                     activation='relu',
                     kernel_regularizer=regularizers.l2(0.001)))##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(p=0.3))##Dropout

classifier.add(Dense(output_dim=10,
                     activation='softmax'))
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])


img_gen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True,
                             rotation_range=10, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.1, zoom_range=0.1,
                             horizontal_flip=True, vertical_flip=False, dtype=np.float32)
img_gen.fit(x_train)



earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) #earlystop

# 開始訓練
classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=100) ,steps_per_epoch=500,
                         epochs=100, validation_data = (x_test, y_test),
                         callbacks = [earlystop])

(50000, 32, 32, 3)










Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100
500/500 [==============================] - 34s 68ms/step - loss: 2.1183 - acc: 0.3487 - val_loss: 1.6148 - val_acc: 0.5068
Epoch 2/100
500/500 [==============================] - 18s 36ms/step - loss: 1.6768 - acc: 0.4783 - val_loss: 1.3805 - val_acc: 0.5801
Epoch 3/100
500/500 [==============================] - 19s 38ms/step - loss: 1.4827 - acc: 0.5397 - val_loss: 1.3097 - val_acc: 0.5897
Epoch 4/100
500/500 [==============================] - 21s 42ms/step - loss: 1.3778 - acc: 0.5681 - val_loss: 1.3264 - val_acc: 0.5836
Epoch 5/100
500/500 [==============================] - 20s 39ms/step - loss: 1.3089 - acc: 0.5878 - val_loss: 1.1448 - val_acc: 0.6405
Epoch 6/100
500/500 [==============================] - 19s 39ms/step - loss: 1.262

Epoch 32/100
500/500 [==============================] - 18s 35ms/step - loss: 1.0465 - acc: 0.6808 - val_loss: 0.9449 - val_acc: 0.7122
Epoch 33/100
500/500 [==============================] - 18s 35ms/step - loss: 1.0471 - acc: 0.6828 - val_loss: 0.9308 - val_acc: 0.7198
Epoch 34/100
500/500 [==============================] - 18s 35ms/step - loss: 1.0423 - acc: 0.6823 - val_loss: 0.9805 - val_acc: 0.7054
Epoch 35/100
500/500 [==============================] - 18s 35ms/step - loss: 1.0363 - acc: 0.6856 - val_loss: 0.9855 - val_acc: 0.6999
Epoch 36/100
500/500 [==============================] - 18s 36ms/step - loss: 1.0338 - acc: 0.6872 - val_loss: 0.9654 - val_acc: 0.7102
Epoch 37/100
500/500 [==============================] - 18s 36ms/step - loss: 1.0386 - acc: 0.6872 - val_loss: 1.0788 - val_acc: 0.6767
Epoch 38/100
500/500 [==============================] - 18s 36ms/step - loss: 1.0355 - acc: 0.6866 - val_loss: 0.9225 - val_acc: 0.7221
Epoch 39/100
500/500 [==========================

500/500 [==============================] - 18s 35ms/step - loss: 0.9827 - acc: 0.7050 - val_loss: 0.9396 - val_acc: 0.7217
Epoch 93/100
500/500 [==============================] - 18s 35ms/step - loss: 0.9759 - acc: 0.7074 - val_loss: 0.9190 - val_acc: 0.7283
Epoch 94/100
500/500 [==============================] - 18s 35ms/step - loss: 0.9689 - acc: 0.7082 - val_loss: 0.9216 - val_acc: 0.7335
Epoch 95/100
500/500 [==============================] - 18s 35ms/step - loss: 0.9734 - acc: 0.7083 - val_loss: 0.9124 - val_acc: 0.7309
Epoch 96/100
500/500 [==============================] - 18s 35ms/step - loss: 0.9715 - acc: 0.7065 - val_loss: 0.9586 - val_acc: 0.7192
Epoch 97/100
500/500 [==============================] - 18s 35ms/step - loss: 0.9686 - acc: 0.7101 - val_loss: 0.8983 - val_acc: 0.7323
Epoch 98/100
500/500 [==============================] - 18s 35ms/step - loss: 0.9723 - acc: 0.7115 - val_loss: 0.8925 - val_acc: 0.7342
Epoch 99/100
500/500 [==============================] - 19s 3